In [2]:
# !pip3 install bert-tensorflow==1.0.1
# !wget https://f000.backblazeb2.com/file/malay-dataset/toxicity/kaum.json
# !wget https://f000.backblazeb2.com/file/malay-dataset/toxicity/weak-learning-toxicity.json
# !wget https://f000.backblazeb2.com/file/malaya-model/pretrained/fastformer-base-social-media-2021-11-02.tar.gz
# !tar -zxf fastformer-base-social-media-2021-11-02.tar.gz
# !rm fastformer-base-social-media-2021-11-02.tar.gz

In [3]:
import os

base = 'https://f000.backblazeb2.com/file/malay-dataset/toxicity/'

urls = """
1. translated-0.json
2. translated-1000000.json
3. translated-1050000.json
4. translated-1100000.json
5. translated-1150000.json
6. translated-1200000.json
7. translated-1450000.json
8. translated-150000.json
9. translated-1500000.json
10. translated-1550000.json
11. translated-1600000.json
12. translated-1650000.json
13. translated-1700000.json
14. translated-1750000.json
15. translated-1800000.json
16. translated-250000.json
17. translated-300000.json
18. translated-350000.json
19. translated-400000.json
20. translated-450000.json
21. translated-50000.json
22. translated-500000.json
23. translated-550000.json
24. translated-600000.json
25. translated-650000.json
26. translated-700000.json
27. translated-750000.json
28. translated-850000.json
29. translated-900000.json
30. translated-950000.json
"""
urls = list(filter(None, urls.split('\n')))
urls = [u.split('. ')[1] for u in urls]
for u in urls:
    print(u)
    os.system(f'wget {base}{u}')

translated-0.json
translated-1000000.json
translated-1050000.json
translated-1100000.json
translated-1150000.json
translated-1200000.json
translated-1450000.json
translated-150000.json
translated-1500000.json
translated-1550000.json
translated-1600000.json
translated-1650000.json
translated-1700000.json
translated-1750000.json
translated-1800000.json
translated-250000.json
translated-300000.json
translated-350000.json
translated-400000.json
translated-450000.json
translated-50000.json
translated-500000.json
translated-550000.json
translated-600000.json
translated-650000.json
translated-700000.json
translated-750000.json
translated-850000.json
translated-900000.json
translated-950000.json


In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [5]:
import random
from unidecode import unidecode
import re

normalized_chars = {}

chars = '‒–―‐—━—-▬'
for char in chars:
    normalized_chars[ord(char)] = '-'

chars = '«»“”¨"'
for char in chars:
    normalized_chars[ord(char)] = '"'

chars = "’'ʻˈ´`′‘’\x92"
for char in chars:
    normalized_chars[ord(char)] = "'"

chars = '̲_'
for char in chars:
    normalized_chars[ord(char)] = '_'

chars = '\xad\x7f'
for char in chars:
    normalized_chars[ord(char)] = ''

chars = '\n\r\t\u200b\x96'
for char in chars:
    normalized_chars[ord(char)] = ' '

    
laughing = {
    'huhu',
    'haha',
    'gagaga',
    'hihi',
    'wkawka',
    'wkwk',
    'kiki',
    'keke',
    'huehue',
    'hshs',
    'hoho',
    'hewhew',
    'uwu',
    'sksk',
    'ksks',
    'gituu',
    'gitu',
    'mmeeooww',
    'meow',
    'alhamdulillah',
    'muah',
    'mmuahh',
    'hehe',
    'salamramadhan',
    'happywomensday',
    'jahagaha',
    'ahakss',
    'ahksk'
}

def make_cleaning(s, c_dict):
    s = s.translate(c_dict)
    return s

def cleaning(string):
    """
    use by any transformer model before tokenization
    """
    string = unidecode(string)
    
    string = ' '.join(
        [make_cleaning(w, normalized_chars) for w in string.split()]
    )
    string = re.sub('\(dot\)', '.', string)
    string = (
        re.sub(re.findall(r'\<a(.*?)\>', string)[0], '', string)
        if (len(re.findall(r'\<a (.*?)\>', string)) > 0)
        and ('href' in re.findall(r'\<a (.*?)\>', string)[0])
        else string
    )
    string = re.sub(
        r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', ' ', string
    )
    
    chars = '.,/'
    for c in chars:
        string = string.replace(c, f' {c} ')
        
    string = re.sub(r'[ ]+', ' ', string).strip().split()
    string = [w for w in string if w[0] != '@']
    x = []
    for word in string:
        word = word.lower()
        if any([laugh in word for laugh in laughing]):
            if random.random() >= 0.5:
                x.append(word)
        else:
            x.append(word)
    string = [w.title() if w[0].isupper() else w for w in x]
    return ' '.join(string)

In [6]:
labels = """
1. severe toxic
2. obscene
3. identity attack
4. insult
5. threat
6. asian
7. atheist
8. bisexual
9. black
10. buddhist
11. christian
12. female
13. heterosexual
14. indian
15. homosexual, gay or lesbian
16. intellectual or learning disability
17. jewish
18. latino
19. male
20. muslim
21. other disability
22. other gender
23. other race or ethnicity
24. other religion
25. other sexual orientation
26. physical disability
27. psychiatric or mental illness
28. transgender
29. white
30. malay
31. chinese
"""
labels = [l.split('. ')[1].strip() for l in labels.split('\n') if len(l)]

In [7]:
import glob

files = glob.glob('translated*')
files

['translated-1750000.json',
 'translated-1450000.json',
 'translated-700000.json',
 'translated-350000.json',
 'translated-600000.json',
 'translated-900000.json',
 'translated-1000000.json',
 'translated-1100000.json',
 'translated-550000.json',
 'translated-150000.json',
 'translated-500000.json',
 'translated-1500000.json',
 'translated-1150000.json',
 'translated-750000.json',
 'translated-850000.json',
 'translated-1650000.json',
 'translated-300000.json',
 'translated-650000.json',
 'translated-950000.json',
 'translated-250000.json',
 'translated-1600000.json',
 'translated-0.json',
 'translated-1550000.json',
 'translated-1800000.json',
 'translated-450000.json',
 'translated-50000.json',
 'translated-1050000.json',
 'translated-1200000.json',
 'translated-1700000.json',
 'translated-400000.json']

In [8]:
import json

X, Y = [], []

for file in files:
    print(file)
    with open(file) as fopen:
        f = json.load(fopen)
        for row in f:
            if len(row[1]) == 29:
                X.append(row[0])
                Y.append(row[1] + [0, 0])
        
    
len(X)

translated-1750000.json
translated-1450000.json
translated-700000.json
translated-350000.json
translated-600000.json
translated-900000.json
translated-1000000.json
translated-1100000.json
translated-550000.json
translated-150000.json
translated-500000.json
translated-1500000.json
translated-1150000.json
translated-750000.json
translated-850000.json
translated-1650000.json
translated-300000.json
translated-650000.json
translated-950000.json
translated-250000.json
translated-1600000.json
translated-0.json
translated-1550000.json
translated-1800000.json
translated-450000.json
translated-50000.json
translated-1050000.json
translated-1200000.json
translated-1700000.json
translated-400000.json


1401054

In [9]:
rejected_labels = ['black', 'white', 'jewish', 'latino']
[labels.index(l) for l in rejected_labels]
labels = [l for l in labels if l not in rejected_labels]

In [10]:
import numpy as np

ydf = pd.DataFrame(np.array(Y))
ydf = ydf.loc[(ydf[8] == 0) & (ydf[28] == 0) & (ydf[16] == 0) & (ydf[17] == 0)]
ydf = ydf.drop([8, 28, 16, 17], axis = 1)
ix = ydf.index.tolist()
Y = ydf.values.tolist()

In [11]:
X = [X[i] for i in ix]
len(X), len(Y)

(1361040, 1361040)

In [12]:
mapping = {'severe_toxic': 'severe toxic', 'identity_hate': 'identity attack',
          'toxic': 'severe toxic', 'melayu': 'malay', 'cina': 'chinese', 'india': 'indian'}

In [13]:
def generate_onehot(tags, depth = len(labels)):
    onehot = [0] * depth
    for tag in tags:
        onehot[labels.index(tag)] = 1
    return onehot

In [14]:
with open('kaum.json') as fopen:
    kaum = json.load(fopen)
    
for k, v in kaum.items():
    print(k, len(v))

melayu 84851
cina 43956
india 20208


In [15]:
with open('weak-learning-toxicity.json') as fopen:
    scores = json.load(fopen)
    
for k, v in scores.items():
    for no in range(len(v)):
        tags = []
        for l, v_ in v[no].items():
            if round(v_) == 1:
                tags.append(mapping.get(l, l))
        tags.append(mapping[k])
        Y.append(generate_onehot(tags))
        X.append(kaum[k][no])

In [16]:
from tqdm import tqdm

for i in tqdm(range(len(X))):
    X[i] = cleaning(X[i])

100%|██████████| 1510055/1510055 [05:44<00:00, 4380.80it/s] 


In [17]:
actual_t, actual_l = [], []

for i in tqdm(range(len(X))):
    if len(X[i]) > 2:
        actual_t.append(X[i])
        actual_l.append(Y[i])

100%|██████████| 1510055/1510055 [00:01<00:00, 1021533.90it/s]


In [18]:
from malaya.text.bpe import WordPieceTokenizer

tokenizer = WordPieceTokenizer('BERT.wordpiece', do_lower_case = False)
# tokenizer.tokenize('halo nama sayacomel')

In [19]:
input_ids, input_masks = [], []
Y = []

maxlen = 150
for i in tqdm(range(len(actual_t))):
    text = actual_t[i]
    tokens_a = tokenizer.tokenize(text)
    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_id)
    
    if len(input_id) <= maxlen:
        input_ids.append(input_id)
        Y.append(actual_l[i])

100%|██████████| 1502318/1502318 [23:20<00:00, 1073.03it/s]


In [20]:
len(input_ids), len(Y)

(1351180, 1351180)

In [21]:
Y[0]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [22]:
import pickle

with open('toxicity-fastformer.pkl', 'wb') as fopen:
    pickle.dump([input_ids, Y], fopen)

In [25]:
!rm *.json *.zip *.csv

rm: cannot remove '*.json': No such file or directory
